USER - USER CF - we can apply covarriance correlation function

In [111]:
import pandas as pd

# Load user data into a pandas DataFrame
user_data = pd.read_csv("user_user_with_toefl_score.csv")

# Apply one-hot encoding on 'univName' and 'major' columns
one_hot_univ = pd.get_dummies(user_data['univName'])
one_hot_major = pd.get_dummies(user_data['major'])

# Concatenate the original DataFrame with the one-hot encoded DataFrames
user_data_encoded = pd.concat([user_data, one_hot_univ, one_hot_major], axis=1)

# Define aggregation functions for grouping by 'userName'
agg_funcs = {col: 'max' for col in one_hot_univ.columns}  # Take max for one-hot encoded university columns
agg_funcs.update({col: 'max' for col in one_hot_major.columns})  # Update for one-hot encoded major columns
for col in user_data.columns:
    if col not in ['userName'] and col not in one_hot_univ.columns and col not in one_hot_major.columns:
        agg_funcs[col] = 'first'  # Take the first value for other non-encoded columns

# Group by 'userName' and apply aggregation functions
user_combined_data = user_data_encoded.groupby('userName').agg(agg_funcs).reset_index()

# Print the combined DataFrame
print(user_combined_data)


            userName  Arizona State University  \
0         ! Superman                         0   
1            !mpulse                         1   
2               $4R4                         1   
3             $aumil                         0   
4            (:)_(:)                         0   
...              ...                       ...   
13384       zulu.sud                         0   
13385       zuperman                         0   
13386    zzkaustavzz                         0   
13387          zztop                         1   
13388  ~rattlesnake~                         0   

       California Institute of Technology  Carnegie Mellon University  \
0                                       1                           0   
1                                       0                           0   
2                                       0                           0   
3                                       0                           1   
4                                 

In [112]:
print(user_combined_data.columns)

Index(['userName', 'Arizona State University',
       'California Institute of Technology', 'Carnegie Mellon University',
       'Clemson University', 'Columbia University', 'Cornell University',
       'George Mason University', 'Georgia Institute of Technology',
       'Harvard University',
       ...
       'toeflScore', 'toeflEssay', 'internExp', 'greV', 'greQ', 'journalPubs',
       'confPubs', 'cgpa', 'univName', 'admit'],
      dtype='object', length=112)


In [113]:
user_data_new = user_combined_data.drop(columns=['univName', 'admit', 'major'])
print(user_data_new[user_data_new['userName'] == "aditya57"])

      userName  Arizona State University  California Institute of Technology  \
2281  aditya57                         0                                   0   

      Carnegie Mellon University  Clemson University  Columbia University  \
2281                           0                   0                    0   

      Cornell University  George Mason University  \
2281                   0                        0   

      Georgia Institute of Technology  Harvard University  ...  researchExp  \
2281                                0                   0  ...            0   

      industryExp  toeflScore  toeflEssay  internExp   greV   greQ  \
2281           35       110.0        28.0        3.0  163.0  166.0   

      journalPubs  confPubs   cgpa  
2281            0         0  0.823  

[1 rows x 109 columns]


In [114]:
import numpy as np
user_data_new.fillna(0, inplace=True)  # Replace NaN values with 0
user_data_new.replace([np.inf, -np.inf], 0, inplace=True)  # Replace infinite values with 0
# Compute Pearson correlation coefficient between each pair of users
correlation_matrix = pd.DataFrame(index=user_data_new['userName'], columns=user_data_new['userName'])

In [115]:
print(user_data_new[user_data_new['userName'] == '143saf'].values.flatten())
print(user_data_new[user_data_new['userName'] == 'AB25'].values.flatten())

['143saf' 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 18 112.0 26.0
 5.0 160.0 167.0 0 0 0.85]
['AB25' 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 66 94.0 21.0
 0.0 146.0 157.0 0 0 0.7828]


In [110]:

for user1 in user_data_new['userName']:
    for user2 in user_data_new['userName']:
#         print(data_user1.shape, data_user2.shape,  user1)
        if user1 != user2:
#             print(user1, user2, user_data_new[user_data_new['userName']== user1].shape,user_data_new[user_data_new['userName']== user2].shape )
#             break
            data_user1 = user_data_new.loc[user_data_new['userName'] == user1, user_data_new.columns != 'userName'].values.flatten()
            data_user2 = user_data_new.loc[user_data_new['userName'] == user2, user_data_new.columns != 'userName'].values.flatten()
#             print(data_user1, user1, data_user2)

            correlation_coefficient, _ = pearsonr(data_user1, data_user2)
            correlation_matrix.loc[user1, user2] = correlation_coefficient

# Filter out NaN values (non-matching users) from the correlation matrix
correlation_matrix = correlation_matrix.dropna()


KeyboardInterrupt: 

In [11]:
# Perform user-user collaborative filtering for a target user
target_user = '143saf'
similar_users = correlation_matrix[target_user]
print("Similar users for", target_user, ":", similar_users)


KeyError: '143saf'